In [1]:
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/MetaTrader_A/MetaTrader 5/terminal64.exe",login=1049732, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
def MetaLink(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_D1, z, y)
    elif x == "H8":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H8, z, y)
    elif x == "H4":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H4, z, y)
    elif x == "H3":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H3, z, y)
    elif x == "H2":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H2, z, y)
    elif x == "H1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H1, z, y)
    elif x == "M30":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M30, z, y)
    elif x == "M15":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M15, z, y)
    elif x == "M5":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M5, z, y)
    elif x == "M1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M1, z, y)
    return data

In [4]:
def MetaLink_range(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_D1, y, z)
    elif x == "H8":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H8, y, z)
    elif x == "H4":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H4, y, z)
    elif x == "H3":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H3, y, z)
    elif x == "H2":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H2, y, z)
    elif x == "H1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H1, y, z)
    elif x == "M30":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M30, y, z)
    elif x == "M15":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M15, y, z)
    elif x == "M5":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M5, y, z)
    elif x == "M1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M1, y, z)
    return data

In [5]:
def MetaLink_pos(p,x,y):
    if x == "D1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_D1, 0, y)
    elif x == "H8":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H8, 0, y)
    elif x == "H4":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H4, 0, y)
    elif x == "H3":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H3, 0, y)
    elif x == "H2":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H2, 0, y)
    elif x == "H1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H1, 0, y)
    elif x == "M30":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M30, 0, y)
    elif x == "M15":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M15, 0, y)
    elif x == "M5":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M5, 0, y)
    elif x == "M1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M1, 0, y)
    return data

In [6]:
def Pip_Screen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            D = d*10000
        elif base == "XAU":
            D = d*100
        elif base == "BTC":
            D = d*10
        elif base == "BCH":
            D = d*100
        elif base == "ETH":
            D = d*10
        elif base == "DSH":
            D = d*100
        elif base == "LTC":
            D = d*100
        elif base == "XMR":
            D = d*100
        elif base == "ZEC":
            D = d*100
        elif base == "ETC":
            D = d*1000
        elif base == "XRP":
            D = d*10000
        else:
            D = d*10000
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            D = d*10
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        D = d*10000
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        D = d*10000
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        D = d*10000
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        D = d*10000
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        D = d*10000
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        D = d*100
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        D = d*10000
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        D = d*1000
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        D = d*1000
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        D = d*10000
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        D = d*1000
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        D = d*1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d*10
    elif quote == "ICrude":
        exchange = 1.00
        D = d*10
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        D = d*10000
        
    return D

In [7]:
def Pip_ReverseScreen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        if base == "XAG":
            D = d/10000
        elif base == "XAU":
            D = d/100
        elif base == "BTC":
            D = d/10
        elif base == "BCH":
            D = d/100
        elif base == "ETH":
            D = d/10
        elif base == "DSH":
            D = d/100
        elif base == "LTC":
            D = d/100
        elif base == "XMR":
            D = d/100
        elif base == "ZEC":
            D = d/100
        elif base == "ETC":
            D = d/1000
        elif base == "XRP":
            D = d/10000
        else:
            D = d/10000
    if quote == "USDt":
        if base == "BTC":
            D = d/10
    elif quote == "AUD":
        D = d/10000
    elif quote == "CAD":
        D = d/10000
    elif quote == "CHF":
        D = d/10000
    elif quote == "EUR": 
        D = d/10000
    elif quote == "GBP":
        D = d/10000
    elif quote == "JPY":
        D = d/100
    elif quote == "NZD":
        D = d/10000
    elif quote == 'NOK':
        D = d/1000
    elif quote == 'SEK':
        D = d/1000
    elif quote == 'SGD':
        D = d/10000
    elif quote == 'DKK':
        D = d/1000
    elif quote == 'MXN':
        D = d/1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d/10
    elif quote == "ICrude":
        exchange = 1.00
        D = d/10
    elif quote == "TRY":
        D = d/10000
        
    return D

In [8]:
def levelgrab(pair):
    
    levslist = []
    
    for p in pair:
        
        S = MetaLink_pos(p,"H1",1000)
        SCM = np.average(abs(S['close'] - S['open']))
        y = savgol_filter(S['close'], 21, 3)
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)           
        levs = tuple(levs)
        levslist.append(levs)

    return levslist, SCM

In [9]:
def levelwheel(levs, SCM):
    
    mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
        
    while mindist < SCM:
        i = 0
        h = -1
        j = 1
        for c in range(0, len(levs)):
            if j == len(levs):
                j = -1
            if abs(levs[i]-levs[h]) < SCM:
                levs[i] = 0
            elif abs(levs[i]-levs[j]) < SCM:
                levs[i] = levs[i] - abs(levs[i]-levs[j])/2
            i += 1
            h += 1
            j += 1
        levs = [v for v in levs if v != 0]
        mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])

    levs = tuple(levs)

    return levs

In [10]:
def Trade_Check(current_trade):
    
    trade_flag = 1
    list_check = 1
    
    to_date=dt.now()
    delta1=timedelta(
        days=0,
        seconds=0,
        microseconds=0,
        milliseconds=0,
        minutes=0,
        hours=1,
        weeks=0
    )
    
    try:
        deals = mt5.history_orders_get(from_date, to_date)
        df = pd.DataFrame(list(deals),columns=deals[0]._asdict().keys())
        df['time_done'] = pd.to_datetime(df['time_done'], unit='s')
        df = df.drop(columns=['ticket', 'magic', 'position_by_id', 'reason', 'time_setup', 'time_setup_msc', 'time_done_msc', 'time_expiration', 'type_time', 'type_filling', 'external_id'])
        aggregation_functions = {'time_done': 'first', 'type': 'first', 'state': 'count', 'volume_initial': 'first', 'volume_current': 'first', 'price_open': 'first', 'sl': 'first', 'tp': 'first', 'price_current': 'first', 'price_stoplimit': 'last', 'symbol': 'first', 'comment': 'first'}
        df = df.groupby(df['position_id']).aggregate(aggregation_functions)
        block = pd.DataFrame(df, columns=['comment', 'state', 'symbol', 'type', 'time_done', 'price_current', 'tp', 'sl'])
        block = block.sort_values(by=['comment', 'time_done'])
        testa = pd.DataFrame(block[(block.comment == "BRUTUS gauge") & (block.state == 2)])
        testa = testa.drop(columns=['comment','state'])
        testa['type'] = np.where(testa['type'] == 1, "SHORT","LONG")
        testa.columns = ['symbol', 'type', 'time', 'entry', 'tp', 'sl']
        testa = testa.set_index('time')
        testa = testa.tz_localize('Etc/GMT-7')
        testa = testa.tz_convert('UTC')
        trade_symb = list(testa['symbol'])
        trade_type = list(testa['type'])
        trades = list(zip(trade_symb,trade_type))

    except IndexError:
        list_check = 0
    
    if list_check == 1:
        for t in trades:
            if t == current_trade:
                trade_flag = 0
        
    return trade_flag

In [11]:
def positioner(a,b,c,d,e):
    
    riskusd = 50.00        

    base, quote = a[:3], a[3:]
    
    ro = 2

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        elif base == "XAU":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BCH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETH":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "DSH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "LTC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "XMR":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ZEC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETC":
            stpips = abs(e - c)*1000
            lot = round(((riskusd*exchange)/10)/10/stpips,ro)
        elif base == "XRP":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        else:
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/stpips,ro)
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        stpips = abs(e - c)*100
        lot = round(((riskusd*exchange)/10)/100/stpips,ro)
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == "entCrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "ICrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
        
    deviation = 20
    
    if b == "LONG":
        price = mt5.symbol_info_tick(a).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS v2",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
    elif b == "SHORT":
        price = mt5.symbol_info_tick(a).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "BRUTUS v2",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
            
    lotflag = "GREEN"
    if lot == 0:
        lotflag = "RED"
    
    return request, lotflag

In [ ]:
def brutus(pair,levslist,SCM):

    ia = 0
    for p in pair:

        
## Call the scope data into the program

        D = MetaLink_pos(p,"H1",100)

    
## Convert to pips and calculate scope trend and scope theta

        STy = Pip_Screen(p,D['close'])
        STx = range(0,len(STy))        
        slope, intercept, r_value, p_value, std_err = stats.linregress(STx,STy)
        ST = slope
        scopetheta = abs(math.degrees(math.atan(ST)))
        
        if scopetheta > 20:
            
            
## Call the main data table A, run a pip screen P, and use the savgol filter S

            A = MetaLink_pos(p,"M5",100)
            S = savgol_filter(A['close'], 21, 3)
        
        
## CHECK 1 : Make sure we have rising or falling peaks

            tops = S[argrelextrema(S, np.greater)[0]]
            bottoms = S[argrelextrema(S, np.less)[0]]
            try:
                if ST > 0:
                    if bottoms[-1] > bottoms[-2]:
                        if bottoms[-2] > bottoms[-3]:
                            check_1 = 1
                        else:
                            check_1 = 0
                    else:
                        check_1 = 0
                elif ST < 0:
                    if tops[-1] < tops[-2]:
                        if tops[-2] < tops[-3]:
                            check_1 = 1
                        else:
                            check_1 = 0
                    else:
                        check_1 = 0
                else:
                    check_1 = 0
            except IndexError:
                check_1 = 0
                break

            if check_1 == 1:
                
                
## Call in the levels and calculate CM

                IT_theta_threshold = 10
    
                entry = S[-2]
                levels = levslist[ia]
                lastone = abs(levels[0] - entry)
                q = 0
                for x in levels:
                    if q == 0:
                        Q = q
                    leveldiff = abs(x - entry)
                    if leveldiff < lastone:
                        Q = q
                    lastone = leveldiff
                    q += 1
                LVL = levels[Q]
                CM_avg = np.average(abs(A['close']-A['open']))
                CM_std = np.std(abs(A['close']-A['open']))
                CM = CM_avg + CM_std

                
## CHECK 2 : Price leaves level with IT > 0 and ITtheta > 20
    
                ch2_range = 7

                if ST > 0:
                    a = -2
                    for x in range(ch2_range):
                        if A['close'][a] <= LVL + CM:
                            IT_o = S[a:-1]
                            IT_x = range(0,len(IT_o))                            
                            slope, intercept, r_value, p_value, std_err = stats.linregress(IT_x,IT_o)
                            IT = slope
                            if IT > 0:
                                ITtheta = abs(math.degrees(math.atan(IT)))
                                if ITtheta > IT_theta_threshold:
                                    check_2 = 1
                                    xi = a
                                    break
                                else:
                                    check_2 = 0
                            else:
                                check_2 = 0
                        else:
                            check_2 = 0
                        a -= 1
                elif ST < 0:
                    a = -2
                    for x in range(ch2_range):
                        if A['close'][a] >= LVL - CM:
                            IT_o = S[a:-1]
                            IT_x = range(0,len(IT_o))                            
                            slope, intercept, r_value, p_value, std_err = stats.linregress(IT_x,IT_o)
                            IT = slope
                            if IT < 0:
                                ITtheta = abs(math.degrees(math.atan(IT)))
                                if ITtheta > IT_theta_threshold:
                                    check_2 = 1
                                    xi = a
                                    break
                                else:
                                    check_2 = 0
                            else:
                                check_2 = 0
                        else:
                            check_2 = 0
                        a -= 1
                else:
                    check_2 = 0

            else:
                check_2 = 0
                    
        
## CHECK 3 : Behind the MOMENT IT TOUCHES THE LEVEL (xi) we want rejection, it must be below LVL + CM

            if check_2 == 1:

                ch3_range = 12

                if ST > 0:
                    a = xi
                    for x in range(ch3_range):
                        if A['close'][a] <= LVL + CM:
                            cndmid = (A['close'][a]+A['open'][a])/2
                            stickmid = (A['high'][a]+A['low'][a])/2
                            if (A['high'][a] - A['low'][a]) == 0:
                                denom = 2
                            else:
                                denom = (A['high'][a] - A['low'][a])
                            midpoint = (cndmid - stickmid)/denom
                            if midpoint >= 0.175:
                                check_3 = 1
                                xi = a
                                break
                            else:
                                check_3 = 0
                        else:
                            check_3 = 0
                        a -= 1
                elif ST < 0:
                    a = xi
                    for x in range(ch3_range):
                        if A['close'][a] >= LVL - CM:
                            cndmid = (A['close'][a]+A['open'][a])/2
                            stickmid = (A['high'][a]+A['low'][a])/2
                            if (A['high'][a] - A['low'][a]) == 0:
                                denom = 2
                            else:
                                denom = (A['high'][a] - A['low'][a])
                            midpoint = (cndmid - stickmid)/denom
                            if midpoint <= -0.175:
                                check_3 = 1
                                xi = a
                                break
                            else:
                                check_3 = 0
                        else:
                            check_3 = 0
                        a -= 1
                else:
                    check_3 = 0

            else:
                check_3 = 0

                
## CHECK 4 : behind the moment we see rejection, we need a negative IT with ITtheta > 20 beginning above the level

            if check_3 == 1:

                ch4_range = 12

                if ST > 0:
                    a = xi
                    for x in range(ch4_range):
                        if A['close'][a] > LVL + CM:
                            IT_o = S[a:xi]
                            IT_x = range(0,len(IT_o))                    
                            slope, intercept, r_value, p_value, std_err = stats.linregress(IT_x,IT_o)
                            IT = slope
                            if IT < 0:
                                ITtheta = abs(math.degrees(math.atan(IT)))
                                if ITtheta > IT_theta_threshold:
                                    check_4 = 1
                                    xi = a
                                    break
                                else:
                                    check_4 = 0
                            else:
                                check_4 = 0
                        else:
                            check_4 = 0
                        a -= 1
                elif ST < 0:
                    a = xi
                    for x in range(ch4_range):
                        if A['close'][a] < LVL - CM:
                            IT_o = S[a:xi]
                            IT_x = range(0,len(IT_o))                            
                            slope, intercept, r_value, p_value, std_err = stats.linregress(IT_x,IT_o)
                            IT = slope
                            if IT > 0:
                                ITtheta = abs(math.degrees(math.atan(IT)))
                                if ITtheta > IT_theta_threshold:
                                    check_4 = 1
                                    xi = a
                                    break
                                else:
                                    check_4 = 0
                            else:
                                check_4 = 0
                        else:
                            check_4 = 0
                        a -= 1
                else:
                    check_4 = 0

            else:
                check_4 = 0
                        
                
## CHECK 5 : behind the down trend, we need an uptrend that started below the level and ended above it

            if check_4 == 1:

                ch5_range = 40

                xii = xi - ch5_range

                if ST > 0:
                    if A['close'][xi] > LVL + CM:
                        if A['close'][xii] < LVL - CM:
                            IT_o = S[xii:xi]
                            IT_x = range(0,len(IT_o))                            
                            slope, intercept, r_value, p_value, std_err = stats.linregress(IT_x,IT_o)
                            IT = slope
                            if IT > 0:
                                ITtheta = abs(math.degrees(math.atan(IT)))
                                if ITtheta > IT_theta_threshold:
                                    check_5 = 1
                                else:
                                    check_5 = 0
                            else:
                                check_5 = 0
                        else:
                            check_5 = 0
                    else:
                        check_5 = 0
                elif ST < 0:
                    if A['close'][xi] < LVL - CM:
                        if A['close'][xii] > LVL + CM:
                            IT_o = S[xii:xi]
                            IT_x = range(0,len(IT_o))    
                            slope, intercept, r_value, p_value, std_err = stats.linregress(IT_x,IT_o)
                            IT = slope
                            if IT < 0:
                                ITtheta = abs(math.degrees(math.atan(IT)))
                                if ITtheta > IT_theta_threshold:
                                    check_5 = 1
                                else:
                                    check_5 = 0
                            else:
                                check_5 = 0
                        else:
                            check_5 = 0
                    else:
                        check_5 = 0
                else:
                    check_5 = 0

            else:
                check_5 = 0
                    
                    
## VERIFICATION OF ALL 5 CHECKS

            if check_1 == 1:
                print(p," CHECK #1: ",check_1)
                print(p," Level:   ",LVL+CM,LVL,LVL-CM)
                if check_2 == 1:
                    print(p," CHECK #2: ",check_2)
                    if check_3 == 1:
                        print(p," CHECK #3: ",check_3)
                        if check_4 == 1:
                            print(p," CHECK #4: ",check_4)
                            if check_5 == 1:
                                #print(p," CHECK #5: ",check_5)
                                proceed = 1
                            else:
                                proceed = 0
                        else:
                            proceed = 0
                    else:
                        proceed = 0
                else:
                    proceed = 0
            else:
                proceed = 0
            
        
        else:
            proceed = 0
                    
                
## Now run the level wheel to refine the levels for TP and SL            

        if proceed == 1:

            levs = levelwheel(levs, SCM)
            levels = levs
            entry = A['close'][-2]
            levels = levslist[ia]
            lastone = abs(levels[0] - entry)
            q = 0
            for x in levels:
                if q == 0:
                    Q = q
                leveldiff = abs(x - entry)
                if leveldiff < lastone:
                    Q = q
                lastone = leveldiff
                q += 1
            CTESTLVL = levels[Q]

            if ST > 0:
                position = 'LONG'
                entry = CTESTLVL
            elif ST < 0:
                position = 'SHORT'
                entry = CTESTLVL
            else:
                position = 0
                entry = 0
                
                
## Check to make sure we aren't placing a duplicate trade

            if position != 0:
                trade_flag = Trade_Check((p,position))
                if trade_flag == 0:
                    position = 0
                    
                    
## Finally, place the trade
                
            if position != 0:

                cend = int(len(levels)-1)
                if position == 'LONG':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex - 1)
                    slindex = int(clindex + 1)
                    if clindex == 0:
                        stoploss = levels[slindex]
                        sldist = entry - stoploss
                        takeprofit = entry + sldist
                    elif clindex == cend:
                        takeprofit = levels[tpindex]
                        tpdist = takeprofit - entry
                        stoploss = entry - tpdist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                elif position == 'SHORT':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex + 1)
                    slindex = int(clindex - 1)
                    if clindex == 0:
                        takeprofit = levels[tpindex]
                        tpdist = entry - takeprofit
                        stoploss = entry + tpdist
                    elif clindex == cend:
                        stoploss = levels[slindex]
                        sldist = stoploss - entry
                        takeprofit = entry - sldist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                else:
                    takeprofit = 0
                    stoploss = 0

                shrink = 0.90
                if position == 'LONG':
                    tpdist = takeprofit - entry
                    takeprofit = entry + tpdist*shrink
                elif position == 'SHORT':
                    tpdist = entry - takeprofit
                    takeprofit = entry - tpdist*shrink

                if position == 'LONG':
                    tpdist = takeprofit - entry
                    sldist = entry - stoploss
                    if sldist > tpdist:
                        stoploss = entry - tpdist
                elif position == 'SHORT':
                    tpdist = entry - takeprofit
                    sldist = stoploss - entry
                    if sldist > tpdist:
                        stoploss = entry + tpdist

                if position == 'LONG':
                    request, lotflag = positioner(p,"LONG",entry,takeprofit,stoploss)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        else:
                            print("Trade Placed:   ",p,"   ","Long","   TP:  ",takeprofit,"   SL:  ",stoploss,)
                elif position == 'SHORT':
                    request, lotflag = positioner(p,"SHORT",entry,takeprofit,stoploss)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        else:
                            print("Trade Placed:   ",p,"   ","Short","   TP:  ",takeprofit,"   SL:  ",stoploss,)
                print(p,position,entry,takeprofit,stoploss,lotflag)

        ia += 1

In [13]:
print("Initializing...")

pair = []
symbols=mt5.symbols_get()
for s in symbols:
    pair.append(s.name)

levslist, SCM = levelgrab(pair)

t2 = 0
while True:
    Drates = MetaLink_pos("EURUSD","M5",1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("Analyzing Markets...")
            brutus(pair,levslist,SCM)
            print("...Analysis Complete")
    t2 = t
    time.sleep(10)

Initializing...
Analyzing Markets...
AUDNZD  CHECK #1:  1
AUDNZD  Level:    1.0832315034280167 1.082848045112766 1.0824645867975153
...Analysis Complete
Analyzing Markets...
AUDNZD  CHECK #1:  1
AUDNZD  Level:    1.083231500182533 1.082848045112766 1.082464590042999
...Analysis Complete
Analyzing Markets...
AUDNZD  CHECK #1:  1
AUDNZD  Level:    1.0832314658420095 1.082848045112766 1.0824646243835225
...Analysis Complete
Analyzing Markets...
AUDNZD  CHECK #1:  1
AUDNZD  Level:    1.0832319282928442 1.082848045112766 1.0824641619326878
EURNOK  CHECK #1:  1
EURNOK  Level:    10.668239953666912 10.662013697286538 10.655787440906163
...Analysis Complete


C:\Users\User\.conda\envs\EnvA\lib\site-packages\scipy\stats\_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Users\User\.conda\envs\EnvA\lib\site-packages\scipy\stats\_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\User\.conda\envs\EnvA\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\User\.conda\envs\EnvA\lib\site-packages\scipy\stats\_stats_mstats_common.py:142: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


Analyzing Markets...
AUDNZD  CHECK #1:  1
AUDNZD  Level:    1.0832324461891891 1.082848045112766 1.0824636440363429
LTCUSD  CHECK #1:  1
LTCUSD  Level:    59.64843184004151 59.45291271657316 59.25739359310481
...Analysis Complete
Analyzing Markets...
AUDNZD  CHECK #1:  1
AUDNZD  Level:    1.0832334226382898 1.082848045112766 1.0824626675872422
EURAUD  CHECK #1:  1
EURAUD  Level:    1.641851845383524 1.641127976462872 1.6404041075422198
USDSGD  CHECK #1:  1
USDSGD  Level:    1.3755450828095033 1.3751623635174692 1.3747796442254352
...Analysis Complete
Analyzing Markets...
AUDNZD  CHECK #1:  1
AUDNZD  Level:    1.0832311235212129 1.082848045112766 1.0824649667043191
EURAUD  CHECK #1:  1
EURAUD  Level:    1.6418490877166618 1.641127976462872 1.640406865209082
USDSGD  CHECK #1:  1
USDSGD  Level:    1.3755403151832293 1.3751623635174692 1.3747844118517092
...Analysis Complete
Analyzing Markets...
EURAUD  CHECK #1:  1
EURAUD  Level:    1.642697039334505 1.6419746355017737 1.6412522316690423


KeyboardInterrupt: 